In [1]:
import requests

### Running API Request

In [3]:
api_file = open("api_key.txt", "r")
key = api_file.read()
res = requests.get("https://api.openaq.org/v2/locations/2178", headers={"X-API-Key": key})

print(res)

### Testing City Pull

In [6]:
url = "https://api.openaq.org/v2/cities?limit=100&page=1&offset=0&sort=asc&order_by=city"
headers = {"accept": "application/json"}
city_list = requests.get(url, headers=headers)

print(city_list.text)

{"meta":{"name":"openaq-api","license":"","website":"/","page":1,"limit":100,"found":4381},"results":[{"country":"DK","city":" ","count":23688,"locations":1,"firstUpdated":"2017-11-09 23:00:00+00","lastUpdated":"2024-04-05 08:00:00+00","parameters":["no2"]},{"country":"JP","city":" ","count":39060950,"locations":1537,"firstUpdated":"2023-07-14 17:00:00+00","lastUpdated":"2024-06-10 01:00:00+00","parameters":["co","no","no2","nox","pm25","so2"]},{"country":"KP","city":" ","count":32997,"locations":4,"firstUpdated":"2024-03-20 08:00:00+00","lastUpdated":"2024-06-10 01:00:00+00","parameters":["co","no2","o3","pm10","pm25","so2"]},{"country":"KR","city":" ","count":6963686,"locations":683,"firstUpdated":"2024-03-20 08:00:00+00","lastUpdated":"2024-06-10 01:00:00+00","parameters":["co","no2","o3","pm10","pm25","so2"]},{"country":"TT","city":" ","count":3879,"locations":1,"firstUpdated":"2024-03-20 01:00:00+00","lastUpdated":"2024-06-10 01:00:00+00","parameters":["co","pm10","pm25"]},{"count

In [8]:
url = "https://api.openaq.org/v2/measurements?date_from=2024-05-30T00%3A00%3A00Z&date_to=2024-06-06T20%3A45%3A00Z&limit=100&page=1&offset=0&sort=desc&radius=1000&order_by=datetime"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{"meta":{"name":"openaq-api","license":"","website":"/","page":1,"limit":100,"found":">100"},"results":[{"locationId":31,"location":"Wekerom-Riemterdijk","parameter":"pm25","value":-999.0,"date":{"utc":"2024-06-06T20:00:00+00:00","local":"2024-06-06T22:00:00+02:00"},"unit":"µg/m³","coordinates":{"latitude":52.11162099964508,"longitude":5.708418999868416},"country":"NL","city":null,"isMobile":false,"isAnalysis":null,"entity":"Governmental Organization","sensorType":"reference grade"},{"locationId":31,"location":"Wekerom-Riemterdijk","parameter":"pm25","value":14.2566,"date":{"utc":"2024-06-06T19:00:00+00:00","local":"2024-06-06T21:00:00+02:00"},"unit":"µg/m³","coordinates":{"latitude":52.11162099964508,"longitude":5.708418999868416},"country":"NL","city":null,"isMobile":false,"isAnalysis":null,"entity":"Governmental Organization","sensorType":"reference grade"},{"locationId":31,"location":"Wekerom-Riemterdijk","parameter":"pm25","value":11.9144,"date":{"utc":"2024-06-06T18:00:00+00:00",

### Returning for a City

In [ ]:
url = "https://api.openaq.org/v2/averages?temporal=day&locations_id=70084&spatial=location&limit=100&page=1"
headers = {"accept": "application/json"}
response = requests.get(url, headers=headers)

print(response.text)